Notebook to actually create item recommendations for each user and each model:

In [1]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from cv_py import CrossValidation
from eda_py import EDA
from recommender_py import Recommender
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares
from eALS_adaptor import eALSAdaptor
from implicit.lmf import LogisticMatrixFactorization
from implicit.bpr import BayesianPersonalizedRanking
import matplotlib.pyplot as plt

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

c:\Users\781110104\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [2]:
dl = DataLoader()
user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [3]:
user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)

user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)

user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [4]:
# get full input data, containing the mapping of item and user ids with the indices used by sparse:

dp_t = DataPreparation(user_item_filtered_log_t)
full_df_t = dp_t._get_full_user_item()

dp = DataPreparation(user_item_filtered_log)
full_df = dp._get_full_user_item()

In [5]:
model_ials_a = AlternatingLeastSquares(factors=150, regularization=90, alpha=0.3, iterations=25)
model_ials_t = AlternatingLeastSquares(factors=100, regularization=90, alpha=1.3, iterations=25)
model_eals_t = eALSAdaptor(factors=50, regularization=50, alpha=0.1, w0=5000, num_iter=25)
model_bpr_a = BayesianPersonalizedRanking(factors=50, regularization=0.03, learning_rate=0.005, iterations=30)
model_lmf_a = LogisticMatrixFactorization(factors=10, regularization=40, iterations=25, neg_prop=50)
model_cosine_a = CosineRecommender()
model_cosine_t = CosineRecommender()

c:\Users\781110104\Anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


In [6]:
model_ials_a.fit(user_item_csr)
model_ials_t.fit(user_item_csr_t)
model_eals_t.fit(user_item_csr_t)
model_bpr_a.fit(user_item_csr)
model_lmf_a.fit(user_item_csr)
model_cosine_a.fit(user_item_csr)
model_cosine_t.fit(user_item_csr_t)

  0%|          | 0/25 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

converting type of user_items to <class 'numpy.float32'>


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

c:\Users\781110104\Anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.03845810890197754 seconds
  warnings.warn(


  0%|          | 0/202000 [00:00<?, ?it/s]

c:\Users\781110104\Anaconda3\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.000997304916381836 seconds
  warnings.warn(


  0%|          | 0/16690 [00:00<?, ?it/s]

In [7]:
rc = Recommender()

In [8]:
recos_ials_a = rc.get_recommendations(model_ials_a, user_item_csr, full_df)
recos_ials_t = rc.get_recommendations(model_ials_t, user_item_csr_t, full_df_t)
recos_eals_t = rc.get_recommendations(model_eals_t, user_item_csr_t, full_df_t)
recos_bpr_a = rc.get_recommendations(model_bpr_a, user_item_csr, full_df)
recos_lmf_a = rc.get_recommendations(model_lmf_a, user_item_csr, full_df)
recos_cosine_a = rc.get_recommendations(model_cosine_a, user_item_csr, full_df)
recos_cosine_t = rc.get_recommendations(model_cosine_t, user_item_csr_t, full_df_t)

In [10]:
common_recos_df_t = rc.get_common_recos_t(recos_ials_t, recos_eals_t, recos_cosine_t)

In [11]:
get_common_recos_a = rc.get_common_recos_a(recos_ials_a, recos_bpr_a, recos_lmf_a, recos_cosine_a)